---
title: "Data Analysis"
subtitle: "Comprehensive Data Cleaning & Exploratory Analysis of Job Market Trends"
execute:
  echo: true   
  warning: false 
jupyter: venv
author:
  - name: Ling Lu
    affiliations:
      - id: U15738240
        name: Boston University
        city: Boston
        state: MA
  - name: Luoyan Zhang
    affiliations:
      - id: U00607539
        name: Boston University
        city: Boston
        state: MA
  - name: Yinuo Wang
    affiliations:
      - id: U78839500
        name: Boston University
        city: Boston
        state: MA
bibliography: references.bib
csl: csl/econometrica.csl
format: 
  html:
    code-fold:  true
    code-tools: true
    toc: true
    number-sections: true
    df-print: paged
---



# Introduction

This section presents a detailed data analysis of job market trends in 2024, focusing on AI-driven changes, salary disparities, and employment trends across different regions and industries.


## Data Import and Cleaning

In [ ]:
# Load necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset


In [ ]:
df = pd.read_csv("lightcast_job_postings.csv")

# Display dataset summary

df.info()
df.describe()

# Data Cleaning & Preprocessing

## Drop Unnecessary Columns

 **Which columns should be dropped, and why?**

The columns selected for removal are considered redundant because they either provide duplicate information, are unnecessary for analysis, or have more detailed equivalents in the dataset. For example, `"ID"` serves as a unique identifier but is often not needed for analysis, while `"URL"` and `"ACTIVE_URLS"` contain job posting links that are useful externally but not critical for data processing. Similarly, `"LAST_UPDATED_TIMESTAMP"` is dropped because `"LAST_UPDATED_DATE"` already provides update information in a more readable format. The `"DUPLICATES"` column, which likely flags repeated entries, is also removed since duplicates can be handled separately.

Additionally, industry and occupational classification columns like `"NAICS2"` to `"NAICS6"` and `"SOC_2"`, `"SOC_3"`, `"SOC_5"` are removed because these represent different levels of classification, and more relevant or updated versions (e.g., `"NAICS_2022_2"` to `"NAICS_2022_6"`) are already present in the dataset. Removing these redundant columns helps streamline the dataset, making it more efficient to analyze without losing valuable information.


In [ ]:
columns_to_drop = [
    "id", "duplicates", "last_updated_timestamp",
    "naics2", "naics3", "naics4", "naics5", "naics6",
    "soc_2", "soc_3", "soc_5"
]

df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=False)

print("Columns after dropping:", df.columns)
df.head()

##  Handle Missing Values
 **How should missing values be handled?**

Missing values should be handled strategically based on their impact on analysis. First, visualizing missing data with a heatmap helps identify patterns and assess severity. Columns with more than 50% missing values are dropped to avoid unreliable or incomplete data. For numerical fields like `"Salary"`, filling missing values with the median ensures the data remains representative without being skewed by outliers. Categorical fields like `"Industry"` are filled with `"Unknown"` to maintain completeness while preserving interpretability. This approach balances data retention and accuracy, ensuring meaningful analysis without introducing bias.


In [ ]:
import missingno as msno
import matplotlib.pyplot as plt

# Visualize missing data
# Identify columns with >10% missing values
missing_threshold = 0.1  
missing_cols = df.columns[df.isnull().mean() > missing_threshold]

# Filter DataFrame
df_missing = df[missing_cols]

# Generate heatmap
plt.figure(figsize=(12, 6))  

msno.heatmap(df_missing)
plt.title("Missing Values Heatmap (Filtered)")
plt.show()


# Drop columns with >50% missing values
df.dropna(thresh=len(df) * 0.5, axis=1, inplace=True)

# Fill missing values

df_original = pd.read_csv("lightcast_job_postings.csv")

df["salary"] = df_original["salary"]

salary_col = "salary" if "salary" in df.columns else None

if salary_col:
    df[salary_col].fillna(df[salary_col].median(), inplace=True)
else:
    print("Warning: No salary-related column found!")


df["naics6_name"].fillna("Unknown", inplace=True)

## Remove Duplicates

To ensure each job is counted only once, we remove duplicates based on job title, company, location, and posting date.


In [ ]:
print("Existing columns in DataFrame:", df.columns.tolist())  # Display actual column names

# Convert column names to lowercase for case-insensitive matching
df.columns = df.columns.str.lower()

columns_to_check = ["title", "company", "location", "posted"]
existing_columns = [col for col in columns_to_check if col in df.columns]

if not existing_columns:
    raise ValueError("None of the specified columns exist in the DataFrame. Check column names!")

print("Before removing duplicates:")
print(df[existing_columns].head())

df = df.drop_duplicates(subset=existing_columns, keep="first")

print("\nAfter removing duplicates:")
print(df[existing_columns].head())

print("\nDuplicates removed based on:", existing_columns)

# Exploratory Data Analysis (EDA)

EDA helps uncover patterns in job postings and salaries across industries. These insights assist job seekers in making informed career decisions.

## Job Postings by Industry
**Why this visualization?**

This bar chart helps identify which industries have the highest number of job postings. It provides insights into industry demand, helping job seekers target sectors with more opportunities.


In [ ]:
import plotly.express as px
import plotly.io as pio

# Set Plotly renderer for Quarto or Jupyter
pio.renderers.default = "iframe"

# Get top 20 industries by job postings
top_n = 20
industry_counts = df["naics6_name"].value_counts().nlargest(top_n).reset_index()
industry_counts.columns = ["Industry", "Count"]

# Create horizontal bar chart with a taller y-axis
fig = px.bar(
    industry_counts,
    x="Industry",
    y="Count",
    title=f"Top {top_n} Job Postings by Industry (NAICS6)",
    labels={"Industry": "Industry", "Count": "Number of Job Postings"}
)

# Extend y-axis and increase figure height
fig.update_layout(
    xaxis_title="Industry",
    yaxis_title="Number of Job Postings",
    yaxis=dict(range=[0, industry_counts["Count"].max() * 1.2]),  # Extend y-axis
    height=1000  # Increase figure height for better spacing
)

fig.show()


**Insights** 

Custom Computer Programming Services, Administrative Management, and Employment Placement Agencies have the highest job postings, indicating strong demand in tech, consulting, and staffing sectors. Computer Systems Design and Commercial Banking also show significant job availability, reflecting growth in IT and finance.
Unclassified Industry has the highest count, which may indicate miscategorized job postings or emerging sectors not yet classified. Other industries like Commercial Banking, Health Insurance, and Educational Services also show moderate job availability.



## Salary Distribution by Industry
**Why this visualization?**

This box plot is used to analyze salary distribution across the top 20 industries. It helps compare median salaries, salary variability, and outliers, which is crucial for understanding income potential in different fields.


In [ ]:
import plotly.express as px

# Get top 20 industries by job postings
top_n = 20
top_industries = df["naics6_name"].value_counts().nlargest(top_n).index

# Filter dataset for top industries
df_filtered = df[df["naics6_name"].isin(top_industries)]

# Create the box plot with an extended y-axis
fig = px.box(
    df_filtered,
    x="naics6_name",
    y="salary",
    title=f"Salary Distribution in Top {top_n} Industries",
    labels={"naics6_name": "Industry", "salary": "Salary ($)"},
    points="all"  # Show all outliers
)

# Extend the y-axis
fig.update_layout(
    xaxis_title="Industry",
    yaxis_title="Salary ($)",
    yaxis=dict(range=[0, df_filtered["salary"].max() * 1.2]),  # Extend y-axis 20% above max salary
    height=1000  # Increase figure height for better visibility
)

fig.show()


**Insights**

Commercial Banking and Tech-related industries show wide salary ranges, indicating opportunities for growth. Temporary Help Services has the lowest pay, reflecting short-term or contract roles. Tech and finance roles offer both high salaries and significant growth potential.



## Remote vs. On-Site Jobs
**Why this visualization?**

This pie chart compares the distribution of remote, hybrid, and on-site jobs, showing workplace flexibility trends. It helps job seekers understand how common remote opportunities are in the current job market.


In [ ]:
fig = px.pie(df, names="remote_type_name", title="Remote vs. On-Site Jobs")
fig.show()

**Insights**

Majority of jobs are not explicitly classified (~78.3%), which may indicate missing or unspecified remote work details in job postings. Only 17% of jobs are fully remote, suggesting that while remote work exists, it is not yet dominant in most industries. Hybrid remote jobs (3.11%) are emerging, but still a small percentage. This indicates a slow transition toward flexible work models. On-site jobs remain the norm (1.58% explicitly labeled as "Not Remote"), reinforcing that many industries still require physical presence. Remote opportunities exist but are limited, meaning job seekers should target specific industries or roles for remote work.

## Geographic Variation of Job Postings
**Why this visualization?**
This map visualizes the number of job postings by U.S. state, offering a clear look at where opportunities are most concentrated geographically. It helps job seekers understand which states have the highest job demand.

```{python}`

state_counts = df["state_name"].value_counts().reset_index()
state_counts.columns = ["State", "Job Postings"]

us_state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND',
    'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# Add abbreviation column
state_counts["State Abbrev"] = state_counts["State"].map(us_state_abbrev)

# Create a choropleth map
fig = px.choropleth(
    state_counts,
    locations="State Abbrev",
    locationmode="USA-states",
    color="Job Postings",
    color_continuous_scale="Blues",
    scope="usa",
    title="Job Postings by U.S. State"
)

fig.update_layout(
    geo=dict(bgcolor='rgba(0,0,0,0)'),
    height=600
)

fig.show()
```
**Insights**
Geographic Insights on Job Postings (2024–2025)

Texas and California are leading in job postings, reflecting strong economies and diverse industries (tech, energy, entertainment).

Florida, New York, and Illinois also show significant job demand due to strong finance, healthcare, and logistics sectors.

Southeastern and Midwestern states like North Carolina, Georgia, and Ohio provide moderate opportunities, often with lower cost of living.

Lower posting states such as Wyoming, Montana, and Alaska may reflect regional economic focus or smaller labor markets.

This distribution suggests job seekers may benefit from targeting high-posting states or considering relocation for greater opportunity.



## Conclusion
This analysis provides insights into the evolving job market in 2024, highlighting AI's impact, salary trends, and employment disparities. The findings indicate how different disciplines benefit from AI's rise, with certain fields seeing more substantial salary growth. Further analysis will explore regional job differences and future career recommendations.